In [19]:
import torch

import src.dnn.datasets
import src.dnn.models
import src.dnn.test
import src.dnn.train
import src.dnn.utils

train_dataset, test_dataset = src.dnn.datasets.DATASETS['MNIST'](train_batch_size=128, test_batch_size=128, path='datasets')
#model = src.dnn.train.train(src.dnn.models.MODELS['LeNet5'](), cuda=True, train_dataset=train_dataset, optimizer_class=torch.optim.Adam, optimizer_args={}, loss=torch.nn.CrossEntropyLoss(), n_epochs=1)
model = src.dnn.utils.init_model(src.dnn.models.MODELS['LeNet5'], model_args={}, use_saved_model=True, model_save_path='models/lenet5.pkl', cuda=True, train_dataset=train_dataset, optimizer_class=torch.optim.Adam, optimizer_args={}, loss=torch.nn.CrossEntropyLoss(), n_epochs=1)
loaded_model = src.dnn.utils.load_model(src.dnn.models.MODELS['LeNet5'](), 'models/lenet5.pkl')
print('Done')

Done


In [20]:
print(dict(model['model'].named_parameters())['fc.f7.bias'])
print(dict(loaded_model.named_parameters())['fc.f7.bias'])
print(src.dnn.test.test(model['model'], test_dataset, loss=torch.nn.CrossEntropyLoss()))
print(src.dnn.test.test(loaded_model, test_dataset, loss=torch.nn.CrossEntropyLoss()))

Parameter containing:
tensor([ 0.0557,  0.0186, -0.0026, -0.1097,  0.0753, -0.0255,  0.0785,  0.0463,
         0.0458,  0.0891], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0557,  0.0186, -0.0026, -0.1097,  0.0753, -0.0255,  0.0785,  0.0463,
         0.0458,  0.0891], device='cuda:0', requires_grad=True)
{'accuracy': 0.9712, 'loss': 0.0007349000092595815}
{'accuracy': 0.9712, 'loss': 0.0007349000092595815}


In [15]:
torch.save(model['model'].state_dict(), 'models/lenet5.pkl')